## Testing out the API connections

In [1]:
import requests
import json

base_uri = f"https://api.nusmods.com/v2/"
acad_year = f"2024-2025"
endpoint = f"/modules/"

module_code = f"CS2105"
query = f"{module_code}.json" 

headers = {"Content-Type": "application/json"}

full_uri = str(base_uri + acad_year + endpoint + query)

resp = requests.get(url=full_uri, headers=headers)
if resp.status_code == 200:
    resp = json.loads(resp.text)
    title = resp["title"]

    # Edge case -> catch modules with no prerequisites
    if "prereqTree" not in resp:
        preReqTree = None
    else:
        preReqTree = resp["prereqTree"]

    # print("Preclusions:", preclusions, "\nPrerequisites:", preReqTree)
elif resp.status_code == 404:
    print("No such module found. Please check module code!")

print(preReqTree)

{'or': ['CS2020:D', 'CS1020:D', 'CS1020E:D', 'CS2030:D', 'CS2030S:D', 'YSC2229:D', 'CS2040:D', 'CS2040S:D', 'CS2040C:D']}


In [2]:
def listifyPrereqs(input : dict) -> list:
    if "or" in input:
        preReqListRaw = input["or"]
        preReqList = []
        for prereq in preReqListRaw:
            if type(prereq) == str:
                preReqList.append(str(prereq).strip(":D"))
            elif type(prereq) == dict: # edge case -> multiple subnodes
                preReqList.append(prereq) 

        return preReqList

print(module_code + " "+ title)
if preReqTree:
    collection = {} # for collecting the respective possible sub-lists
    if "and" in preReqTree:
        elements = preReqTree['and'] # get sub-branches
        count = 1
        for value in elements:
            if type(value) == dict:
                collection[count] = listifyPrereqs(value)
            else:
                collection[count] = str(value).strip(":D")
            count += 1

    result = listifyPrereqs(preReqTree)
    if result != None:
        try:
            finalList = []
            if "and" in str(result):
                for ele in result:
                    if type(ele) == dict:
                        fragment = ele['and'] # get sub-branches
                        collection2 = {}
                        count = 1
                        for value in fragment:
                            if type(value) == dict:
                                collection2[count] = listifyPrereqs(value)
                            else:
                                collection2[count] = str(value).strip(":D")
                            count += 1
                        finalList.append(collection2)

                     # normal string elements (no subnodes)
                    else:
                        finalList.append(ele)
                print(finalList)
            else:
                print(result)
        except:
            print(result)
    elif collection != {}:
        print(collection)
    else:
        singlePRQ = [str(preReqTree).strip(":D")]
        print(singlePRQ)
else:
    print("No prerequisities!")

CS2105 Introduction to Computer Networks
['CS2020', 'CS1020', 'CS1020E', 'CS2030', 'CS2030S', 'YSC2229', 'CS2040', 'CS2040S', 'CS2040C']


## Actually doing some stuff with the data


In [3]:
import pandas as pd
import requests
import json

df = pd.read_excel("NUS Module Planning.xlsx", sheet_name="Graduating Requirements", skiprows=16, nrows=26 - 16)
df = df.dropna(axis=1, how='any')
df

,ID Mod 2,Unnamed: 2,CD Course 1,4,Prereqs
0,CS1231S,N.A.,Discrete Structures (Math),4,"['MA1301X', 'MA1301']"
1,CS2030,N.A.,Programming Methodology 2,4,"['CS1010', 'CS1010E', 'CS1010X', 'CS1101S', 'C..."
2,CS2040C,N.A.,Data Structures and Algorithms,4,"['CS1010', 'CS1010E', 'CS1010X', 'CS1101S', 'C..."
3,CS2100,N.A.,Computer Organization,4,"['YSC1212', 'CS1010', 'CS1010J', 'CS1010E', 'C..."
4,CS2101,N.A.,Effective Communication for Computing Professi...,4,No prerequisites
5,CS2103T,N.A.,Software Engineering,4,"['CS2020', 'CS1020', 'CS1020E', {1: ['CS2030',..."
6,CS2105,N.A.,Intro to Computer Networks,4,"['CS2020', 'CS1020', 'CS1020E', 'CS2030', 'CS2..."
7,CS2106,N.A.,Intro to OS,4,"['CS2100', 'EE2007', 'EE2024', 'EE2028']"
8,CS2107,N.A.,Intro to Infosec,4,"['YSC1212', 'CS1010J', 'CS1101S', 'CS1010', 'C..."
9,CS3235,N.A.,Computer Security,4,"{1: ['CS2105', 'EE3204', 'EE4204'], 2: ['CS210..."


In [4]:
def listifyPrereqs(input : dict) -> str:
     if "or" in input:
        preReqListRaw = input["or"]
        preReqList = [str(prereq).strip(":D") for prereq in preReqListRaw]
        preReqList = []
        for prereq in preReqListRaw:
            if type(prereq) == str:
                preReqList.append(str(prereq).strip(":D"))
            elif type(prereq) == dict: # edge case -> multiple subnodes
                preReqList.append(prereq) 

        return preReqList

def get_prerequisites(module_code : str):
    base_uri = "https://api.nusmods.com/v2/"
    acad_year = "2024-2025"
    endpoint = "/modules/"
    query = f"{module_code}.json"
    headers = {"Content-Type": "application/json"}
    full_uri = base_uri + acad_year + endpoint + query

    resp = requests.get(url=full_uri, headers=headers)
    if resp.status_code == 200:
        resp_json = json.loads(resp.text)
        
        if "prereqTree" not in resp_json:
            return "No prerequisites"
        
        preReqTree = resp_json["prereqTree"]
        
        if preReqTree:
            collection = {} # for collecting the respective possible sub-lists
            if "and" in preReqTree:
                elements = preReqTree['and'] # get sub-branches
                count = 1
                for value in elements:
                    if type(value) == dict:
                        collection[count] = listifyPrereqs(value)
                    else:
                        collection[count] = str(value).strip(":D")
                    count += 1
            
            result = listifyPrereqs(preReqTree)
            if result != None:
                try:
                    finalList = []
                    if "and" in str(result):
                        for ele in result:
                            if type(ele) == dict:
                                fragment = ele['and'] # get sub-branches
                                collection2 = {}
                                count = 1
                                for value in fragment:
                                    if type(value) == dict:
                                        collection2[count] = listifyPrereqs(value)
                                    else:
                                        collection2[count] = str(value).strip(":D")
                                    count += 1
                                finalList.append(collection2)

                            # normal string elements (no subnodes)
                            else:
                                finalList.append(ele)
                        return(str(finalList))
                    else:
                        return(str(result))
                except:
                    return(str(result))
            elif collection != {}:
                return str(collection)
            else:
                singlePRQ = [str(preReqTree).strip(":D")]
                return str(singlePRQ)
        else:
            return "No prerequisites"
    elif resp.status_code == 404:
        return "No such module found. Please check module code!"
    
df['Prereqs'] = df['ID Mod 2'].apply(get_prerequisites)
df

,ID Mod 2,Unnamed: 2,CD Course 1,4,Prereqs
0,CS1231S,N.A.,Discrete Structures (Math),4,"['MA1301X', 'MA1301']"
1,CS2030,N.A.,Programming Methodology 2,4,"['CS1010', 'CS1010E', 'CS1010X', 'CS1101S', 'C..."
2,CS2040C,N.A.,Data Structures and Algorithms,4,"['CS1010', 'CS1010E', 'CS1010X', 'CS1101S', 'C..."
3,CS2100,N.A.,Computer Organization,4,"['YSC1212', 'CS1010', 'CS1010J', 'CS1010E', 'C..."
4,CS2101,N.A.,Effective Communication for Computing Professi...,4,No prerequisites
5,CS2103T,N.A.,Software Engineering,4,"['CS2020', 'CS1020', 'CS1020E', {1: ['CS2030',..."
6,CS2105,N.A.,Intro to Computer Networks,4,"['CS2020', 'CS1020', 'CS1020E', 'CS2030', 'CS2..."
7,CS2106,N.A.,Intro to OS,4,"['CS2100', 'EE2007', 'EE2024', 'EE2028']"
8,CS2107,N.A.,Intro to Infosec,4,"['YSC1212', 'CS1010J', 'CS1101S', 'CS1010', 'C..."
9,CS3235,N.A.,Computer Security,4,"{1: ['CS2105', 'EE3204', 'EE4204'], 2: ['CS210..."


In [5]:
from openpyxl import load_workbook

wb = load_workbook('NUS Module Planning.xlsx')
sheet = wb['Graduating Requirements']

if 'Prereqs' not in [cell.value for cell in sheet[17]]:
    sheet.cell(row=17, column=8, value='Prereqs')

for idx, value in enumerate(df['Prereqs'], start=18):
    sheet.cell(row=idx, column=8, value=value)

wb.save('NUS Module Planning.xlsx')